In [0]:
import random
from yaml import safe_load
from pyspark.sql import functions as F

with open('../params.yml', 'r') as f:
  params = safe_load(f)

CATALOG = params['data_params']['catalog']
SCHEMA = params['data_params']['schema']


In [0]:
from pyspark.sql.functions import col, row_number, lit, current_timestamp, monotonically_increasing_id, rand, when
from pyspark.sql.types import StringType
from pyspark.sql.window import Window
import random

devices = spark.table(f"{CATALOG}.{SCHEMA}.tower_devices").select(['tower_id', 'device_id', 'device']).orderBy(rand())



# Add a row number column to uniquely identify rows
window_spec = Window.orderBy(monotonically_increasing_id())
devices_with_row_num = devices.withColumn("row_num", row_number().over(window_spec))

# Randomly select between 3-25 rows to have 'Telemetry Issue'
num_issues = random.randint(3, 25)

print(num_issues)
devices_with_status = devices_with_row_num.withColumn(
    "status",
    when(F.col("row_num").between(1, num_issues), "Telemetry Issue").otherwise("System Online and Functioning")
).withColumn('event_timestamp', current_timestamp()).drop("row_num")

devices_with_status.display()
devices_with_status.write.mode('append').insertInto(f"{CATALOG}.{SCHEMA}.device_status")


In [0]:
print(f"{CATALOG}.{SCHEMA}.device_status")